# Main functions used in experiments

In [15]:
import numpy as np
from collections import defaultdict
from tqdm import tqdm as _tqdm

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer
%matplotlib inline
import matplotlib.pyplot as plt
import sys

import random
import time
assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

## Environment: Windy gridworld
Gives a reward of -1 for each step taken, while the final state is not reached

In [16]:
from windy_gridworld import WindyGridworldEnv
env = WindyGridworldEnv()
env??

## Policy

### Target policy (choose greedy vs non-greedy)
Greedy policy 

In [17]:
class GreedyPolicy(object):
    """
    A simple epsilon greedy policy.
    """
    def __init__(self, Q):
        self.Q = Q
    
    def get_probs(self, states, actions):
        """
        This method takes a list of states and a list of actions and returns a numpy array that contains 
        a probability of perfoming action in given state for every corresponding state action pair. 

        Args:
            states: a list of states.
            actions: a list of actions.

        Returns:
            Numpy array filled with probabilities (same length as states and actions)
        """   
        
        # Inefficient but kept same structure as below if we change policy later
        probs = [1 if a == np.argmax(self.Q[s]) else 0 for s,a in zip(states, actions)]
        
        return probs
        
    def sample_action(self, obs):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            obs: current state

        Returns:
            An action (int).
        """

        # find out what the max action is
        best_action = np.argmax(self.Q[obs])
        
        return best_action

In [31]:
class EpsilonGreedyPolicy(object):
    """
    A simple epsilon greedy policy.
    """
    def __init__(self, Q, epsilon):
        self.Q = Q
        self.epsilon = epsilon
        
    def get_probs(self, states, actions):
        # loop over the state action lists and compute probabilities according to eps greedy
        probs = [1-self.epsilon if a == np.argmax(Q[s]) else self.epsilon/self.Q.shape[0] for s, a in zip(states, actions)]
                
        return probs
        
    
    def sample_action(self, obs):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            obs: current state

        Returns:
            An action (int).
        """
         
        best_actions = [i for i, j in enumerate([self.Q[obs][i] for i in range(4)]) 
                   if j == max([self.Q[obs][i] for i in range(4)])] 
        p = np.random.uniform()
        if p > self.epsilon:
            # choose one of the best actions
            action = np.random.choice(best_actions)
        else:
            # return a random action
            action = np.random.randint(0,4)
                
        return action


### Behavioural policy
Random policy from blackjack lab. 
TODO: experiment with behavioural policies to check which yield interesting results

In [32]:
class RandomPolicy(object):
    """
    A behavioural policy
    """
    def __init__(self, nS, nA):
        self.probs = np.ones((nS, nA)) * 1/nA
        
    def get_probs(self, states, actions):
        """
        This method takes a list of states and a list of actions and returns a numpy array that contains 
        a probability of perfoming action in given state for every corresponding state action pair. 

        Args:
            states: a list of states.
            actions: a list of actions.

        Returns:
            Numpy array filled with probabilities (same length as states and actions)
        """        
        probs = [self.probs[s,a] for s,a in zip(states, actions)]
        
        return probs

    
    def sample_action(self, state):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            state: current state

        Returns:
            An action (int).
        """
        p_s = self.probs[state]
        
        return np.random.choice(range(0,self.probs.shape[1]), p=p_s)

In [33]:
random_policy = RandomPolicy(env.nS, env.nA)

## Monte Carlo

## Sampling function given an env and policy
Function to sample an episode from the env.

In [34]:
def sample_episode(env, policy):
    """
    A sampling routine. Given environment and a policy samples one episode and returns states, actions, rewards
    and dones from environment's step function and policy's sample_action function as lists.

    Args:
        env: OpenAI gym environment.
        policy: A policy which allows us to sample actions with its sample_action method.

    Returns:
        Tuple of lists (states, actions, rewards, dones). All lists should have same length. 
        state after the termination is not included in the list of states.
    """
    # initialize
    states = []
    actions = []
    rewards = []
    dones = []
    
    # get a starting state
    s = env.reset()
    d = False
    
    # keep looping until done, don's save the terminal state
    while not d:
        states.append(s)
        a = policy.sample_action(s)
        s, r, d, _ = env.step(a)
        
        # save                
        actions.append(a)
        rewards.append(r)
        dones.append(d)
        

    return states, actions, rewards, dones

In [22]:
# check the length of episodes that are generated for random policy
for episode in range(10):
    trajectory_data = sample_episode(env, random_policy)
#     print("Episode {}:\nStates {}\nActions {}\nRewards {}\nDones {}\n".format(episode,*trajectory_data))
    print(f"length of episode {episode}: {len(trajectory_data[0])}")


length of episode 0: 7393
length of episode 1: 9911
length of episode 2: 170
length of episode 3: 12086
length of episode 4: 509
length of episode 5: 10901
length of episode 6: 3895
length of episode 7: 4957
length of episode 8: 9287
length of episode 9: 6468


In [36]:
# check the length of episodes that are generated for eps greedy policy
Q = np.zeros((env.nS, env.nA))
bp = EpsilonGreedyPolicy(Q, epsilon=0.1)

for episode in range(10):
    trajectory_data = sample_episode(env, bp)
#     print("Episode {}:\nStates {}\nActions {}\nRewards {}\nDones {}\n".format(episode,*trajectory_data))
    print(f"length of episode {episode}: {len(trajectory_data[0])}")

length of episode 0: 7401
length of episode 1: 5997
length of episode 2: 8393
length of episode 3: 499
length of episode 4: 3313
length of episode 5: 6857
length of episode 6: 2296
length of episode 7: 2545
length of episode 8: 26983
length of episode 9: 5788


### TO-DO: MC Ordinary Importance Sampling (make it work for windy gridworld)
Status: updated to update Q instead of V

In [10]:
def mc_ordinary_importance_sampling(env, behavior_policy, target_policy, num_episodes, discount_factor=1.0,
                           sampling_function=sample_episode):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given target policy using behavior policy and weighted importance sampling.
    
    Args:
        env: OpenAI gym environment.
        behavior_policy: A policy used to collect the data.
        target_policy: A policy which value function we want to estimate.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        sampling_function: Function that generates data from one episode.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of current V and count of returns for each state
    # to calculate an update.
    Q = defaultdict(lambda: defaultdict(float))
    returns_count = defaultdict(lambda: defaultdict(float))
    
    # sample episodes
    for i in tqdm(range(num_episodes), position=0):
        states, actions, rewards, dones = sampling_function(env, behavior_policy)
        
        # extract target and behavioral probabilities
        target_probs = target_policy.get_probs(states, actions)
        behavioral_probs = behavior_policy.get_probs(states, actions)

        G = 0        
        
        # loop backwards over the trajectory
        for timestep in range(len(states)-1, -1, -1):
            s = states[timestep]
            r = rewards[timestep]
            a = actions[timestep]
            G = discount_factor * G + r
            
            returns_count[s][a] += 1 

            # compute the ratio using the two probability lists
            ratio = np.prod([t/b for t, b in zip(target_probs[timestep:], behavioral_probs[timestep:])])

            # use every visit incremental method
            Q[s][a] += 1/returns_count[s][a] * (ratio * G - Q[s][a])
        
    return Q

In [11]:
Q = np.zeros((env.nS, env.nA))
bp = EpsilonGreedyPolicy(Q, epsilon=0.05)
gp = GreedyPolicy(Q)
Q_10k = mc_ordinary_importance_sampling(env, bp, gp, num_episodes=10)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(Q_10k)

### MC: Weighted Importance Sampling

##### (TODO: Eventually: merge the two functions into one with a weighted flag)

In [46]:
def mc_weighted_importance_sampling(env, behavior_policy, target_policy, num_episodes, discount_factor=1.0,
                           sampling_function=sample_episode):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given target policy using behavior policy and ordinary importance sampling.
    
    Args:
        env: OpenAI gym environment.
        behavior_policy: A policy used to collect the data.
        target_policy: A policy which value function we want to estimate.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        sampling_function: Function that generates data from one episode.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # create a matrix defaultdict for the Q function and the sum of weights C
    Q = defaultdict(lambda: defaultdict(float))
    C = defaultdict(lambda: defaultdict(float))
    
    # sample episodes
    for i in tqdm(range(num_episodes), position=0):
        states, actions, rewards, dones = sampling_function(env, behavior_policy)
        
        # extract target and behavioral probabilities
        target_probs = target_policy.get_probs(states, actions)
        behavioral_probs = behavior_policy.get_probs(states, actions)
        
        # initialize the return and the weight
        G = 0
        W = 1
        
        # loop backwards over the trajectory
        for timestep in range(len(states)-1, -1, -1):            
            # extract info of current timestep from trajectory    
            s = states[timestep]
            r = rewards[timestep]
            a = actions[timestep]
            G = discount_factor * G + r
            
            # add W to the sum of weights C
            C[s][a] += W
            
            # update Q function incrementally
            Q[s][a] += W/C[s][a] * (G - Q[s][a])
            
            # update the weight
            W *= (target_probs[timestep])/(behavioral_probs[timestep])
            
            # break out of the loop if the weights are 0
            if W == 0:
                break
            
    return Q

In [47]:
Q = np.zeros((env.nS, env.nA))
bp = EpsilonGreedyPolicy(Q, epsilon=0.05)
gp = GreedyPolicy(Q)
Q_10k = mc_weighted_importance_sampling(env, bp, gp, num_episodes=200)

100%|██████████| 200/200 [01:45<00:00,  1.90it/s]

In [48]:
print(dict(Q_10k))

{48: defaultdict(<class 'float'>, {3: -1.0, 2: -2.0005725004522805, 1: -5.012202043274311, 0: -39.0}), 68: defaultdict(<class 'float'>, {0: -2.0006118702835, 3: -3.0, 1: -4.0028652595420615, 2: -4.0214810627473145}), 69: defaultdict(<class 'float'>, {3: -3.000613189564369, 1: -4.006833008299926, 2: -4.655265492176622, 0: -4.00000108038424}), 59: defaultdict(<class 'float'>, {2: -4.007392831869315, 1: -4.024237261368661, 3: -3.001838149294274, 0: -3.0094529119628386}), 49: defaultdict(<class 'float'>, {2: -4.013772815655844, 1: -3.0021101369959573, 3: -2.0005413288157397, 0: -4.013838303570058}), 39: defaultdict(<class 'float'>, {2: -3.003545659157099, 3: -6.004279394477442, 1: -4.004442904649776, 0: -5.003703219690444}), 29: defaultdict(<class 'float'>, {2: -4.006719726445993, 0: -6.007699012048301, 3: -7.000047812834603, 1: -5.02377780148853}), 19: defaultdict(<class 'float'>, {2: -5.005950520848916, 3: -10.575218191794857, 0: -8.743976508556646, 1: -6.132756369026636}), 9: defaultdic

## Temporal Difference

### TO-DO: TD Ordinary Importance Sampling (make it work for gridworld)
Copied from TD_lab. Currently on-policy, needs to be off-policy.

Confused: do we need value functions instead of q-values? Do we even use importance weights in off-policy TD? Are there more off-policy TD methods besides SARSA?

In [12]:
def sarsa(env, policy, Q, num_episodes, discount_factor=1.0, alpha=0.5):
    """
    SARSA algorithm: On-policy TD control. Finds the optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        policy: A policy which allows us to sample actions with its sample_action method.
        Q: Q value function, numpy array Q[s,a] -> state-action value.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        
    Returns:
        A tuple (Q, stats).
        Q is a numpy array Q[s,a] -> state-action value.
        stats is a list of tuples giving the episode lengths and returns.
    """
    
    # Keeps track of useful statistics
    stats = []
    
    for i_episode in tqdm(range(num_episodes)):
        i = 0
        R = 0
        
        # initial state is 3,0 in the grid (according to source code)
        s = env.reset()
        a = policy.sample_action(s)
        final_state_reached = False
        
        while True:
            # new actions
            s_prime, r, final_state, _ = env.step(a)
            
            # keep track of stats
            R += r
            i += 1    
            
            # sample action at state s_prime
            a_prime = policy.sample_action(s_prime)

            # update Q 
            Q[s][a] += alpha * (r + discount_factor * Q[s_prime][a_prime] - Q[s][a])    
    
            # update policy
            policy.Q = Q
            
            # if final state, terminate loop
            if final_state:
                break
        
            # update current s and a for next iteration
            s = s_prime
            a = a_prime
            
        stats.append((i, R))
        
    episode_lengths, episode_returns = zip(*stats)
    return Q, (episode_lengths, episode_returns)

### TO-DO: TD Weighted Importance Sampling (same as above but weighted)

In [ ]:
## TD weighted importance sampling

## Experiments